Проверояная #7 (по рекомендательным системам) - https://goo.gl/forms/8kNZlOFQYRbMgOzU2

## Матричные разложения

$T$ - множество тем (топиков, интересов и т.п.), $|T| << |U|$, $|T| << |I|$

$p_{tu}$ - неизвестный профиль клиента, $P=(p_{tu})_{|T|\times|U|}$

$q_{ti}$ - неизвестный профиль продукта, $Q=(q_{tu})_{|T|\times|I|}$

Задача: $r_{ui}=\sum_{t\in T}\pi_t p_{tu}q_{ti}$

$R=P^T\Delta Q$, $\Delta=diag(\pi_1,\dots,\pi_{|T|})$ - матричная запись

$p(u,i)=\sum_{t\in T} p(t)\cdot p(u|t)\cdot q(i|t)$ - вероятностный смысл

Методы поиска профилей:
* SVD - сингулярное разложение
* NMF - неотрицательное матричное разложение
* PLSA - вероятностный латентный семантический анализ

### Градиентный спуск

Задача - $R\sim P\times Q^T=\hat R$

$\hat r_{ij}=p^T_i q_j=\sum_{k=1}^K p_{ik}q_{kj}$

$e^2_{ij}=(r_{ij}-\hat r_{ij})^2=(r_{ij}-\sum_{k=1}^{K}p_{ik}q_{kj})^2$

$\frac{\partial }{\partial p_{ik}} e_{ij}^2=-2e_{ij}q_{kj}$

$\frac{\partial }{\partial q_{kj}} e_{ij}^2=-2e_{ij}p_{ik}$

$p'_{ik}=p_{ik}+\alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2=p_{ik}+2\alpha e_{ij}q_{kj}$

$q'_{kj}=q_{kj}+\alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2=q_{kj}+2\alpha e_{ij}p_{ik}$

$E=\sum_{(u_i,d_j,r_{ij})\in T}e_{ij}=\sum_{(u_i,d_j,r_{ij})\in T}(r_{ij}-\sum_{k=1}^{K} p_{ik}q_{kj})^2$

In [5]:
import numpy as np

In [6]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T

In [7]:
R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
    ]
R = np.array(R)
N = len(R)
M = len(R[0])
K = 2

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)

nR = np.dot(nP, nQ.T)
nR

array([[ 5.00090396,  2.91921985,  4.5218978 ,  0.99735541],
       [ 3.95841926,  2.31768995,  3.7574694 ,  0.99657152],
       [ 1.08390728,  0.79330032,  5.06316337,  4.96189952],
       [ 0.95805419,  0.6872052 ,  4.11969453,  3.97248012],
       [ 1.9458653 ,  1.25938915,  4.9000108 ,  4.03828649]])

### SVD

Задача - $\|R - U^T\Delta V\| \rightarrow min_{U,V,\Delta}$

In [8]:
#создаем матрицу оценок фильмов
X=np.zeros((6, 7))
X = np.array([[4,4,5,0,0,0,0], [5,5,3,4,3,0,0], [0,0,0,4,4,0,0], [0,0,0,5,4,5,3], [0,0,0,0,0,5,5], [0,0,0,0,0,4,4]])

<img src="../figures/matr1.png" alt="Matr" style="width: 200px;"/>

Находим сингулярное разложение, смотрим на получающиеся матрицы

In [9]:
# ваш код
U,s, V = np.linalg.svd(X)

In [10]:
#превращаем вектор весов матрицу весов
S = np.zeros((6, 7))

In [11]:
S[:6,:6]=np.diag(s)
S

array([[  1.26226575e+01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   1.06683922e+01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   7.29730704e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.64107896e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   9.53988881e-01,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   3.30004874e-17,
          0.00000000e+00]])

In [12]:
#пробуем восстанавливать исходную матрицу
np.dot(U, np.dot(S, V))

array([[  4.00000000e+00,   4.00000000e+00,   5.00000000e+00,
          9.04398596e-16,   9.20373504e-16,   5.87028089e-16,
          5.95694946e-16],
       [  5.00000000e+00,   5.00000000e+00,   3.00000000e+00,
          4.00000000e+00,   3.00000000e+00,  -6.02155430e-16,
         -2.84405778e-16],
       [  3.57220994e-16,   7.13285194e-17,   4.31472232e-16,
          4.00000000e+00,   4.00000000e+00,  -1.87363603e-15,
         -1.32959913e-15],
       [  1.47433400e-15,   9.91229891e-16,   2.29874676e-15,
          5.00000000e+00,   4.00000000e+00,   5.00000000e+00,
          3.00000000e+00],
       [  7.09455292e-16,   3.63118858e-16,   6.09097584e-16,
         -1.14174352e-15,  -1.38769170e-15,   5.00000000e+00,
          5.00000000e+00],
       [  6.07200061e-16,   3.49176919e-16,   8.32138391e-16,
         -8.10873899e-16,  -8.13583095e-16,   4.00000000e+00,
          4.00000000e+00]])

In [13]:
Y=np.dot(U, np.dot(S, V))

In [14]:
#вычисляем ошибку приближения со всеми факторами
np.linalg.norm(X-Y)

1.3677535415069609e-14

In [15]:
#обнуляем 5-ый фактор
S[4,4]=0

In [16]:
Y=np.dot(U, np.dot(S, V))

In [17]:
np.linalg.norm(X-Y)

0.95398888134339599

In [16]:
#обнуляем 4-ый фактор
S[3,3]=0
Y=np.dot(U, np.dot(S, V))
np.linalg.norm(X-Y)

1.8982188885974189

In [17]:
from sklearn.decomposition import NMF

In [18]:
#инициализируем праметры неотрицательной матричной факторизации с тремя факторами
model = NMF(n_components=3, init='random', random_state=0)

In [19]:
#применяем модель к матрице оценок фильмов
model.fit(X)

NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=3, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [20]:
#извлекаем компоненты 
W = model.fit_transform(X);
H = model.components_;

In [21]:
#зрители-факторы
W

array([[ 2.24793179,  0.        ,  0.        ],
       [ 2.23118348,  1.66417342,  0.        ],
       [ 0.        ,  1.91022987,  0.        ],
       [ 0.        ,  2.17779586,  1.98547926],
       [ 0.        ,  0.        ,  2.58505415],
       [ 0.        ,  0.        ,  2.06804332]])

In [22]:
#факторы-зрители
H

array([[ 1.97582315,  1.97582315,  1.78771163,  0.06530907,  0.        ,
         0.        ,  0.        ],
       [ 0.08820566,  0.08820566,  0.        ,  2.22716484,  1.91238711,
         0.18750422,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.01994477,  0.        ,
         2.03430582,  1.82223255]])

In [23]:
#восстанавливаем исходную матрицу
Y=np.dot(W,H)
Y

array([[ 4.44151567,  4.44151567,  4.0186538 ,  0.14681033,  0.        ,
         0.        ,  0.        ],
       [ 4.55521349,  4.55521349,  3.98871265,  3.85210505,  3.18254379,
         0.31203953,  0.        ],
       [ 0.16849309,  0.16849309,  0.        ,  4.2543968 ,  3.65309897,
         0.35817615,  0.        ],
       [ 0.19209393,  0.19209393,  0.        ,  4.88991029,  4.16478872,
         4.44741791,  3.61800494],
       [ 0.        ,  0.        ,  0.        ,  0.0515583 ,  0.        ,
         5.2587907 ,  4.71056983],
       [ 0.        ,  0.        ,  0.        ,  0.04124664,  0.        ,
         4.20703256,  3.76845586]])

In [24]:
#находим суммарную ошибку
np.linalg.norm(X-Y)

2.0790165776696892

In [153]:
X-Y

array([[ -4.17630451e-01,  -4.17630451e-01,   9.10134534e-01,
         -1.77553482e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  4.32786107e-01,   4.32786107e-01,  -9.43834651e-01,
          1.85489943e-01,  -2.83877488e-01,  -9.16166928e-07,
         -9.16166467e-07],
       [ -3.49851161e-01,  -3.49851161e-01,   0.00000000e+00,
         -1.63275040e-01,   2.49544104e-01,  -1.83431827e-02,
         -1.00144815e-02],
       [ -1.02989387e-01,  -1.02989387e-01,   0.00000000e+00,
          1.14281347e-03,  -6.10132784e-04,   7.58800160e-05,
          4.12016364e-05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -4.98384306e-05,   0.00000000e+00,  -5.50766951e-06,
         -2.99057651e-06],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -3.98707445e-05,   0.00000000e+00,  -4.40613561e-06,
         -2.39246121e-06]])

In [140]:
# на точность приближения влияют параметры модели (например, можно увеличить число факторов)